In [22]:
using Random
using Distributions
using StatsBase
using FLoops

include("main.jl")

In [25]:
"""
DATA CONFIG
"""
alpha = 0.05

# data
B  = 1    # num. coverage probabilities per boxplot
S  = 1   # num. samples per coverage probability
nx = 25      # size of group 1
ny = 13     # size of group 2
dtype = Float32
seed = 123

"""
POPULATION SETTINGS
"""

Random.seed!(seed)
distrTypeX = LogNormal{dtype}
muX = dtype.(round.(rand(Uniform(0, 1), B), digits=4))
sdX = 2 * muX
paramsX = (muX, sdX)

distrTypeY = LogNormal{dtype}
muY = dtype.(round.(rand(Uniform(0, 1), B), digits=4))
sdY = 2 * muY
paramsY = (muY, sdY)

mc_size = try
    (binomial(nx+ny, nx) > 45_000) ? 10_000 : 0
catch
    0
end
    
@time main(B, S, nx, ny, distrTypeX, paramsX, distrTypeY, paramsY,
           mc_size=mc_size, dtype=dtype, seed=seed, save_csv=false)

  0.275636 seconds (967.58 k allocations: 499.807 MiB, 4.04% gc time)
